In [209]:
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt

In [210]:
import requests

# rate limiting is important to avoid accidental service abuse of the OpenFDA API provider
from ratelimit import limits, sleep_and_retry

# cache API calls in a sqllite file to reduce the number of requests to openfda server
import requests_cache
requests_cache.install_cache('openfda_cache')

OPENFDA_API = "https://api.fda.gov/drug/event.json"

@sleep_and_retry
@limits(calls=40, period=60)
def call_api(params):
    """
    OpenFDA API call. Respects rate limit. Overrides default data limit
    Input: dictionary with API parameters {search: '...', count: '...'}
    Output: nested dictionary representation of the JSON results section
    
    OpenFDA API rate limits:
         With no API key: 40 requests per minute, per IP address. 1000 requests per day, per IP address.
         With an API key: 240 requests per minute, per key. 120000 requests per day, per key.
    """
    if not params:
        params = {}
    params['limit'] = params.get('limit', 1000)
    response = requests.get(OPENFDA_API, params=params)
    print(response.url)

    if response.status_code != 200:
        raise Exception('API response: {}'.format(response.status_code))
    return response.json()['results']

OPENFDA_METADATA_YAML = "https://open.fda.gov/fields/drugevent.yaml"
# munch is a yaml parser with javascript-style object access
from munch import Munch

def api_meta():
    """
    YAML file with field description and other metadata retrieved from the OpenFDA website
    Parses YAML file and provides syntactic sugar for accessing nested dictionaries
    Example: .patient.properties.patientagegroup.possible_values.value
    Note: reserved words, such as count and items still have to be accessed via ['count'], ['items']
    """
    response = requests.get(OPENFDA_METADATA_YAML)
    if response.status_code != 200:
        raise Exception('Could not retrieve YAML file with drug event API fields')
    y = Munch.fromYAML(response.text)
    return y['properties']

In [211]:
api_meta().patient.properties.keys()

dict_keys(['drug', 'patientagegroup', 'patientdeath', 'patientonsetage', 'patientonsetageunit', 'patientsex', 'patientweight', 'reaction', 'summary'])

In [212]:
# For example, 20100729 is 07/29/2010"
#start_date = input("Enter the beginning of your desired date range: " )
#end_date = input("Enter the end of your desired date range: ")

#country_list = input("Enter the countries you would like to limit your search to: ")


In [213]:
#print(start_date, end_date)
start_date = '20110801'
end_date = '20110806'

#api_key = 'rioyXUSAMBQfuxPWqw8Qgf7bR0Bi96eaRNwxb92Q'
columbia_api_key = 'Og4jAa0KIhPJkiwaxXVD6VHp3DGqoQf37JFPeRct'


test_date_str = 'receiptdate:[' + start_date + ' TO ' +  end_date +']'
search_field = test_date_str + ' AND primarysource.reportercountry: "FRANCE+FR"'
country_search = 'primarysource.reportercountry:"FRANCE" AND "FR"' 

#'search': "receivedate:[20040101 TO 20200101] AND patient.reaction.reactionmeddrapt.exact: {}".format(


print(test_date_str)
#print(search_field)


test_out = call_api({
    'api_key': columbia_api_key,
    'limit': 1000,
    'search': test_date_str + ' AND ' + country_search

})


receiptdate:[20110801 TO 20110806]
https://api.fda.gov/drug/event.json?api_key=Og4jAa0KIhPJkiwaxXVD6VHp3DGqoQf37JFPeRct&limit=1000&search=receiptdate%3A%5B20110801+TO+20110806%5D+AND+primarysource.reportercountry%3A%22FRANCE%22+AND+%22FR%22


In [214]:
testDF=pd.DataFrame(test_out)

#for item in test_out:
#    print(item)

results_obj = open('results.txt', 'w')
results_obj.write(str(testDF))

#results_obj2 = open('results2.txt', 'w')
#for item in test_out:
#    results_obj2.write(item)


6536